# 5 Nonlinear Dynamics

So far we have only simulated linear models, which are of course only a small subset of all possible dynamic models. As you have seen in the lectures and tutorials, solving models can become much more complicated when non-linearities are involved, and is often restricted to local analyses of linearised systems around fixed points. Simulating non-linear models however is is not so different to simulating linear models. The only complication is that the fixed point problem that arises in implicit timestepping methods may not always be straightforward to solve.

In this tutorial you will learn how to circumvent the problem of unsolvable fixed points and we apply it to the Crank-Nicolson method. In the following, we will introduce the Runge-Kutta timestepping scheme, which is a popular method used in many software applications due its excellent convergence properties.

Moreover, we will touch upon numerical methods to find fixed points of generic dynamic models using Newton's method.

In [ ]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colormaps

from matplotlib import rc
rc("text", usetex=True) # Latex font in figures

### Solow model with constant technology and population

Consider Solow's growth model, defined by the following equations:

$\begin{align}
    Y &= AK^{\alpha}L^{(1-\alpha)} \hspace{0.1cm};\hspace{0.5cm}0<\alpha<1\\
    Y &= C + I \\
    S &= sY = I\\
    K' &= I - \gamma K = sY
\end{align}$

The first equation is a standard Cobb-Douglas production function with technology parameter $A$, capital input $K$, and labour input $L$, which defines the total output $Y$. The second equation states that all production is either consumed or invested, and the third equation is the savings-investment identity $S=I$. Note that a fixed share of total income $s$ is saved in every period. Finally, the law of motion states that the rate of change of capital equals investments (positive inflow to capital) minus depreciation (outflow from capital, through wear-and-tear or similar), which is a linear function of capital with the depreciation rate $\gamma$.

In per-capita terms and with a technology parameter $A=1$, the last equation can be restated as

$\begin{equation}
    k' = sk^{\alpha} - \gamma k
\end{equation}$

Solving the fixed-point problem of the implicit Crank-Nicolson method to iterate this equation forward in time would be cumbersome at best. Instead, we will approximate $k_{t+\delta}$ with an explicit Euler step, and calculate the rate of change at this point, so we can use it in our Crank-Nicolson scheme. This is not as accurate as the analytical solution, but still improves a lot on the explicit Euler scheme applied by itself. Note also the similiarity to Runge's method, where we take half an Euler step, and calculate the derivative at the point $t+\delta/2$.

##### EXERCISE

Implement a single step of the Crank-Nicolson and Runge methods in the functions below. For the Crank-Nicolson method, estimate $k_{t+\delta}$ with an explicit Euler step and then calculate the derivates at $k_t$ and $k_{t+\delta}$ to perform the step forward. 

Write a separate function for the Euler step, which you can use in the other functions. That way, we can compare behaviour of the methods afterwards, and your code becomes more readable. Generally, it is a good idea to split functions into subroutines, so that each function has narrow and focused responsibilities.

##### SOLUTION

Recall the **Crank-Nicolson** method: $k_{t+\delta} \approx k_t + \frac{\delta}{2}(k'_t + k'_{t+\delta})$. Inserting the derivatives would yield the equation

$\begin{equation}
    (1+\delta\gamma)k_{t+\delta} - \delta sk_{t+\delta}^{\alpha} = (1- \delta\gamma)k_t
\end{equation}$

which we do not want to solve by hand. Instead, we estimate $k'_{t+\delta}$ with an explicit Euler step and approximate the derivative at this point: 

$\begin{align}
    \tilde{k}_{t+\delta} &\approx k_t + \delta(sk^{\alpha} - \gamma k)\\
    \tilde{k'}_{t+\delta} &\approx s\tilde{k}_{t+\delta}^{\alpha} - \gamma\tilde{k}_{t+\delta}
\end{align}$ 

Finally, we use the derivatives at both points to implement the time step:

$\begin{equation}
    k_{t+\delta} \approx k_t + \frac{\delta}{2}(k'_t + \tilde{k'}_{t+\delta})
\end{equation}$

For **Runge's method** using the central difference quotient, nothing changes compared to linear models: we take half an Euler step, calculate the derivative at this point and use that rate of change to calculate $k(t+\delta)$.

In [ ]:
# euler step
def ee_solow_step(k, alpha, s, gamma, delta):
    # TO DO

# crank-nicolson
def cn_solow_step(k, alpha, s, gamma, delta):
    # TO DO

def runge_solow_step(k, alpha, s, gamma, delta):
    # TO DO

##### EXERCISE

Implement the function that simulates Solow's model with either of the two timestepping functions!

Inputs are the standard model parameters, the time increment $\delta$, as well as the total number of *unit* time steps `T` and the timestepping function. Output should be a numpy array of capital values $k$.

In [ ]:
def simulate_Solow(k_0, alpha, s, gamma, delta, T, timestep_func):
    # total number of time increments: inverse of increment times unit steps
    T = int(T/delta)
    
    # time series of capital, as an array
    k_ts = np.empty(T+1)
    k_ts[0] = k_0
    
    for t in range(T):
        # TO DO
        
    return k_ts

In [ ]:
# set variable values
delta = 1/16  # time increment
T = 75        # time unit steps
alpha = 1/3   # Cobb-Douglas exponent
gamma = 0.1   # depreciation rate of capital
s = 0.1       # savings rate
k_0 = 0.1     # initial capital

Solow_T_CN = simulate_Solow(k_0, alpha, s, gamma, delta, T, cn_solow_step)

In [ ]:
plt.figure(figsize=(8,4))
plt.title('Solow growth model implementation')

# x-axis re-adjusted
t = np.linspace(0, T, int(T/delta)+1)
# plot the time series
plt.plot(t, Solow_T_CN, lw=0.8)

# analytical solution of the steady state:
ss = (s / gamma) ** (1 / (1-alpha))
# horizontal line at steady state
plt.hlines(ss, 0, T, ls='--', lw=0.5, color="black", alpha=0.8)

# axis labels
plt.xlabel("t")
plt.ylabel("k")

### Higher-order methods

So far we have used four different timestepping methods of differential equations of the general form $y'(t) = f(y(t), t)$:

$\begin{align}
    y(t+\delta)\ &\approx y(t) + \delta f(y(t), t) \tag{Explicit Euler} \\
    y(t+\delta)\ &\approx y(t) + \delta f(y(t+\delta), t+\delta) \tag{Implicit Euler} \\
    y(t+\delta)\ &\approx y(t) + \delta f(y(t) + \delta/2(f(y(t), t), t + \delta/2) \tag{Runge} \\
    y(t+\delta)\ &\approx y(t) + \delta/2[f(y(t), t) + f(y(t+\delta), t+\delta)]  \tag{Crank-Nicolson}  
\end{align}$

We noticed that the methods of Runge and Crank-Nicolson worked much better, as they did not simply extrapolate using the derivative at one of the endpoints. As you may have guessed, using methods with even more intermediate steps can indeed improve convergence further. 

#### Runge-Kutta

A standard method is the "classic Runge-Kutta" method. It uses a weighted average of four different derivatives:

$y(t+\delta) = y(t) + \frac{\delta}{6}(k_1 + 2k_2 + 2k_3 + k_4)$

with

$\begin{align}
    k_1 &= f(y(t), t+\delta) \\
    k_2 &= f(y(t) + \delta\frac{k_1}{2}, t+\frac{\delta}{2}) \\
    k_3 &= f(y(t) + \delta\frac{k_2}{2}, t+\frac{\delta}{2}) \\
    k_4 &= f(y(t) + \delta k_3, t+\delta)
\end{align}$

That is, $k_1$ is the slope at the beginning of the interval (Euler step), $k_2$ is the slope at the midpoint of the interval. This midpoint is calculated using $k_1$, i.e. the slope at the beginning of the interval (we would apply this slope in the Runge's central difference method). $k_3$ is also the slope at the midpoint, but now we calculate it using $k_2$, i.e. the estimate of the midpoint slope using the slope we had previously calculated using the slope at the beginning of the interval. $k_4$ is the slope at the endpoint of the interval, calculated using $k_3$. So ultimately, we extrapolate from the rate of change in the beginning ($k_1$) to calculate the midpoint, we use that estimate to calculate the slope at this midpoint ($k_2$), and then re-estimate the slope at the midpoint ($k_3$) with this initial estimate of the slope at this point. Finally, we use the improved midpoint slope estimate to estimate the endpoint and calculate the slope at that value of $y$ ($k_4$). We do not discard any of the slope estimates, as we would in case of the Runge method, where we only use the slope we calculated in the second step, but instead we take a weighted average of all the slopes.

##### EXERCISE

In order to compare convergence to the true value, we will once more revisit the cobweb model $p' = 45 - 3p$. Implement one step of the classig Runge-Kutta method for this model.

##### SOLUTION

We calculate the slopes sequentially, as we need the value of the last one to obtain the next. I.e. we calculate the rate of change of the price at its current value, $p'$ using the given equation: $k_1 = 45 - 3p$. Then, we implement the equation at the next point, which we calculate with this slope: $k_2 = 45 - 3(p+0.5\delta k_1)$ etc.

In [ ]:
def rk_1step(p, delta):
    # TO DO

A few of the previous methods, for comparison

In [ ]:
# one explicit method
def exp_euler_1step(p, delta):
    return p + delta * (45 - 3*p)

# one implicit method
def cn_1step(p, delta):
    denominator = 2 + 3 * delta
    return ((2 - 3*delta) / denominator) * p + (90 / denominator) * delta

analytical solution

In [ ]:
def cobweb_1period(p_0, T):
    return (p_0 - 15) * np.exp(-3*T) + 15

Simulation over $T$ steps

In [ ]:
def simulate_cobweb(p, T, n, timestep_func):
    delta = 1/n
    for t in range(n*T):
        p = timestep_func(p, delta)
    return p

Evaluation of convergence

In [ ]:
p0 = 60
T = 1
p_analytical = cobweb_1period(p0, T)

# errors
ee_error = cn_error = rk_error = 1

# analyse convergence, doubling the number of steps per time unit
for exponent in range(2,9):
    n = 2 ** exponent
    # update last period errors
    ee_last_error = ee_error
    cn_last_error = cn_error
    rk_last_error = rk_error
    
    # starting prices
    p_ee = p_ie = p_rg = p_cn = p_rk = p0
    
    # calculate solutions
    p_ee = simulate_cobweb(p_ee, T, n, exp_euler_1step)
    p_cn = simulate_cobweb(p_cn, T, n, cn_1step)
    p_rk = simulate_cobweb(p_rk, T, n, rk_1step)
    
    # new errors
    ee_error = p_ee - p_analytical
    cn_error = p_cn - p_analytical
    rk_error = p_rk - p_analytical
    
    # convergence
    print(f'delta = 1/{n}:')
    print(
        f'Explicit Euler scheme \n current value: p = {p_ee:.4e};\t error = {ee_error:.3e};\t error ratio = {ee_last_error / ee_error:.3}'
    )
    print(
        f'Crank-Nicolson scheme \n current value: p = {p_cn:.4e};\t error = {cn_error:.3e};\t error ratio = {cn_last_error / cn_error:.3}'
    )
    print(
        f'Runge-Kutta scheme \n current value: p = {p_rk:.4e};\t error = {rk_error:.3e};\t error ratio = {rk_last_error / rk_error:.3}'
    )
    print('\n')

Note how the error ratio of the Runge-Kutta method converges to 16. That means that as we take time increments of half the size, the errors are only a sixteenth of their previous size. Even though a few extra steps of computation are necessary, the improvement in convergence means we can reach the same level of accuracy at a lower computational cost.

##### BONUS EXERCISE

Implement the logistic growth function $x' = rx(1-x)$ using any method of your choice!

### Systems of non-linear equations

Consider the system of Tutorial 5, exercise 1:

$\begin{align}
    z_1' &= z_2(z_1 + 1)\\
    z_2' &= z_1(z_2 + 3)
\end{align}$

Implementing simulation methods for systems of non-linear equations should bring no surprises to you: instead of calculating the derivative for one variable, we calculate the derivatives of all state variables, and increment all variables according to their respective laws of motion.

##### EXERCISE
Write four separate functions:
* one that takes the current state vector (numpy array) as input and returns the vector of derivatives at that point (also as a numpy array),
* one that implements a single explicit Euler step,
* one that implements a single Crank-Nicolson step,
* and finally, one that implements a single step of the Runge-Kutta method.

##### SOLUTION

In [ ]:
def derivative(z):
    # TO DO
def ee_step(z, delta):
    # TO DO

def cn_step(z, delta):
    # TO DO

def rk_step(z, delta):
    # TO DO

Finally, we can simulate this system too.  Let's try different initial conditions to see how it does not always reach the same steady state. Some initial conditions lead to explosive behaviour, some sonverge so a finite steady state. As the explosive ones diverge very fast, we simulate them for shorter time periods. Try out different timestepping functions, so you can verify that they all lead to the same (hopefully correct) result!

In [ ]:
delta = 1/2014       # time increment
step_func = rk_step  # define a new name here, so it is easier to change the method for every point at once

# two different time lengths for stable and unstable starting positions (for plotting purposes)
T_stable = 2
T_unstable = 1

# different initial conditions
# stable
za = np.array([1, -2]) 
zb = np.array([-4, -4])
zc = np.array([-0.75, 2])
# unstable
zd = np.array([2, -1.52])
ze = np.array([-0.5, 2])

z1 = np.empty((2,int(T_stable/delta) + 1))
z1[:, 0] = za

z2 = np.empty((2,int(T_stable/delta) + 1))
z2[:, 0] = zb

z3 = np.empty((2,int(T_stable/delta) + 1))
z3[:, 0] = zc

z4 = np.empty((2,int(T_unstable/delta) + 1))
z4[:, 0] = zd

z5 = np.empty((2,int(T_unstable/delta) + 1))
z5[:, 0] = ze

U_stable = int(T_stable/delta)
U_unstable = int(T_unstable/delta)

for u in range(U_stable):
    z1[:, u+1] = step_func(z1[:, u], delta)
    z2[:, u+1] = step_func(z2[:, u], delta)
    z3[:, u+1] = step_func(z3[:, u], delta)
    
for u in range(U_unstable):
    z4[:, u+1] = step_func(z4[:, u], delta)
    z5[:, u+1] = step_func(z5[:, u], delta)

In [ ]:
plt.figure()

for z in [z1, z2, z3, z4, z5]:
    plt.plot(z[0], z[1], lw=0.7, c='black')
    # add arrow heads
    dz1 = z[0,-1] - z[0,-2]
    dz2 = z[1, -1] - z[1, -2]
    plt.arrow(z[0,-2], z[1, -2], dz1, dz2, head_width=0.075)

# steady states
plt.scatter([0, -1],[0, -3], s=15, c='black')

# give the figure a more "mathy" feel by removing the top and left spines, 
# add arrowheads to the others and make them the z1 and z2 axes of the coordinate system
ax = plt.gca()
ax.spines['bottom'].set_position('zero')
ax.spines['left'].set_position('zero')
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
arrow_fmt = dict(markersize=4, color='black', clip_on=False)
ax.plot((1), (0), marker='>', transform=ax.get_yaxis_transform(), **arrow_fmt)
ax.plot((0), (1), marker='^', transform=ax.get_xaxis_transform(), **arrow_fmt)

plt.xlabel("$z_1$", loc="right", labelpad=-10, fontsize=12)
plt.ylabel("$z_2$", loc="top", labelpad=-10, fontsize=12)

plt.grid(ls=':', lw=0.4)

This figure shows the saddle stability of the steady state in the origin quite nicely. 

A good alternative to phase diagrams for non-linear systems are gradient fields. Those draw little arrows at discrete points of the grid, representing the direction the system takes if it starts from there. We can implement it with the derivatives function above:

In [ ]:
# gridpoints on both axes
z1_ax = np.linspace(-3, 2, 20)
z2_ax = np.linspace(-4, 3, 20)

# create a 3d grid: one dimension for each axis
# 1 dimension because derivatives come in two values (dz1, dz2)
grid = np.empty((2, len(z1_ax), len(z2_ax)))

# fill grid
for i, z1_ in enumerate(z1_ax):
    for j, z2_ in enumerate(z2_ax):
        grid[:,i,j] = derivative(np.array([z1_, z2_]))
        
# colours will be based on magnitude of change
# create a grid with derivatives where total change is scaled between 0 and 1
col_grid = np.empty((len(z1_ax), len(z2_ax)))
for i in range(len(z1_ax)):
    for j in range(len(z2_ax)):
        col_grid[i,j] = np.sqrt(grid[0,i,j]**2 + grid[1,i,j]**2)
# scaling with logistic function - created better distribution of hot and cold colours in my opinion
col_grid = 1 / (1 + np.exp(-0.5*(col_grid - col_grid.mean())))

In [ ]:
plt.figure()

cmap = colormaps['coolwarm']

for i, z1_ in enumerate(z1_ax):
    for j, z2_ in enumerate(z2_ax):
        dz1, dz2 = grid[:, i, j]
        # scale the length of the arrow
        length = np.sqrt(dz1**2 + dz2**2)
        dz1_scaled = 0.15 * dz1 / length
        dz2_scaled = 0.15 * dz2 / length
        
        # color according to the magnitude of change
        col = cmap(col_grid[i,j])

        plt.arrow(
            z1_, z2_, dz1_scaled, dz2_scaled, 
            color=col, lw=0.7, head_width=0.035, length_includes_head=True
        )

# steady states
plt.scatter([0, -1],[0, -3], s=15, c='black')

plt.grid(lw=0.3, ls=':', color="black", alpha=0.4)

Note how I scaled the magnitudes of change (length of the vectors) by the logistic function

$\begin{equation}
    \tilde{x} = \frac{1}{1 + exp(-0.5(x-\bar{x}))}
\end{equation}$

The primary reason was to rescale the lengths so that they lie within $\tilde{x}\in[0,1]$, because that can be used by the colormap I am applying to create a colour scheme. I could have used a more simple regularization method, such as

$\begin{equation}
    \tilde{x} = \frac{x - min(x)}{max(x) - min(x)}
\end{equation}$

However, upon trying it out, I found the distribution of colours quite uninformative - try it yourself, if you are interested. The logistic scaling created a more interesting colour scheme.

The "coolwarm" colormap provides a quite intuitive scheme for magnitudes of change, but if you have issues such as colour blindness, feel free to try some alternatives ("viridis", "jet", "hot", "gist_heat", "gnuplot", "rainbow", "plasma", "inferno", "terrain", "ocean")

### Finding fixed points

Another common problem in many numerical applications is the detection of fixed points or determining roots (zeros). Other examples include optimisation, i.e. finding values of a variable at which some function has a zero-derivative. Here, the roots of our differential equation of the form $x' = f(x,t) = 0$ represent the steady states of a dynamic model, so this is a good opportunity to look into it.

One of the simplest, yet still a powerful method is Netwon's method. The method is iterative, i.e. we start from a random "guess" and then update that guess, try from that updated guess again, and so on. The specific updating rule is as follows:

$\begin{equation}
    x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}
\end{equation}$

Intuitively that means that we follow the tangent line of the function at our current guess and set our next guess to be the root of that tangent line. See the figure below to illustrate this updating mechanism. It should be intuitively clear, why this generally takes us towards the roots of a function, at least most of the time. This step can be performed step over and over again to approximate the roots of a function, without having to calculate it by hand. We have to implement a stopping criterion of course, e.g. we break out of this process once we have found a value at which the function is close enough to zero.

In the case of higher dimensions we have to replace derivatives with the Jacobian matrix, but otherwise it is the same. Consider a system of $n$ state variables, denoted as a vector $\mathbf{x}$, and the dynamics are described by the vector-valued function $F: \mathbb{R}^n\rightarrow\mathbb{R}^n$, i.e. $\mathbf{x}' = F(\mathbf{x})$. The updating scheme to find $\mathbf{x}'=0$ is then

$\begin{equation}
    \mathbf{x}_{n+1} = \mathbf{x}_n - J_F(\mathbf{x_n})^{-1}F(\mathbf{x_n})
\end{equation}$

where $J_F(\mathbf{x_n})$ is the Jacobian of $F$ at $\mathbf{x}_n$.



In [ ]:
plt.figure()
plt.title("One step of Newton's method")
ax = plt.gca()

# the function of which we want to find the roots
def func(x):
    return 0.5*x**3 - 0.5

x0 = 2.5

x = np.linspace(-1, 3, 1000)
y = func(x)
# tangent slope at x=2
slope = 1.5 * x0**2
intercept = func(x0) - slope * x0
tangent = slope*x[625:] + intercept

# plot the function and the tangent at point x0
plt.plot(x, y, lw=0.7, c='black', label='function')
plt.plot(x[625:], tangent, lw=0.7, c='red', label='tangent')

# vertical line to indicate x0
plt.vlines(x0, 0, func(x0), color='black', lw=0.7, ls='--')

# this is the equation to find the next value of x
# here we use it to find the point for an accurate graphical representation
x1 = x0 - func(x0) / slope

# custom ticks and ticklabels to include x_0 and x_1
xticks_major = list(range(-1, 4)) 
xticks_minor = [x0, x1]
xticklabels = ["$x_0$", "$x_1$"]
plt.xticks(xticks_major)
ax.set_xticks(ticks=xticks_minor, labels=xticklabels, minor=True)

### stylistic stuff

# text to findicate the value f(x0)
plt.text(2.2, 7.5, "$f(x_0)$")

# grid lines
plt.grid(lw=0.3, ls=':')

# give the figure a more "mathy" feel by removing the top and left spines, 
# add arrowheads to the others and make them the z1 and z2 axes of the coordinate system
ax.spines['bottom'].set_position('zero')
ax.spines['left'].set_position('zero')
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
arrow_fmt = dict(markersize=4, color='black', clip_on=False)
ax.plot((1), (0), marker='>', transform=ax.get_yaxis_transform(), **arrow_fmt)
ax.plot((0), (1), marker='^', transform=ax.get_xaxis_transform(), **arrow_fmt)

##### EXERCISE

Consider Solow's model of economic growth as an example again: $k' = f(k) = sk^{\alpha} - \gamma k$. We know how to find the steady states of the model, but let us implement Newton's method to let the computer find it for us. Fill out the remaining parts of the functions below, which implements one iteration of the method:

In [ ]:
def k_prime(k, alpha, s, gamma):
    '''
    calculate the rate of change of capital given the current amount of capital k' = f(k)
    '''
    # TO DO

def k_doubleprime(k, alpha, s, gamma):
    '''
    calculate the derivative of the rate of change of capital k'' = f'(k)
    '''
    # TO DO

def newton_solow_1step(k, alpha, s, gamma):
    # TO DO

##### EXERCISE

Implement the remaining parts of the iterative procedure in the function below. Inputs `k`, `alpha`, `s`, `gamma` are well-known by now. `max_iter` specifies the maximum number of iterations we perform before "giving up" and `crit` is the error that we consider small enough to be satisfied with the convergence.

In [ ]:
def newton_solow(k, alpha, s, gamma, max_iter=1000, crit=1e-6):
    for i in range(max_iter): # even if we do not converge, we have to stop eventually
        # TODO: 
        #    - newton step
        #    - check if we have converged after every step
        #    - repeat
        
    # finally, we return our estimate
    return k

In [ ]:
alpha = 0.3   # Cobb-Douglas exponent
gamma = 0.1   # depreciation rate of capital
s = 0.2       # savings rate
k0 = 0.5      # initial capital

print(f'starting from initial value k0 = {k0}')
print(f'steady state found at k={round(newton_solow(k0, alpha, s, gamma),4)}\n')

k0 = 0.1
print(f'starting from initial value k0 = {k0}')
print(f'steady state found at k={round(newton_solow(k0, alpha, s, gamma),4)}\n')

Note how the function actually has two steady states: the one you already know, and the one in the origin. Multiple steady states often occur in non-linear systems, and since initial guesses are typically random, it is important to try different starting points!

##### BONUS EXERCISE

Implement Newton's method on the 2-dimensional system: 

$\begin{align}
    z_1' &= z_2(z_1 + 1)\\
    z_2' &= z_1(z_2 + 3)
\end{align}$

### Bonus: Numerical differencing

Sometimes the derivative of a function may not be easy to calculate, or you don't know the exact function of the dynamic model you are analysing. That can be particularly relevant when you are working with empirical data for which you do not know the exact data-generating function. In that case, numerical differencing comes in handy. Consider the definition of derivatives:

$f'(x) = \underset{h\rightarrow0}{lim}\frac{f(x+h) - f(x)}{h}$. 

We cannot actually implement infinitesimally small steps, but we can approximate derivatives with finite differences, using finitely small values of $h$:

$f'(x)\approx\frac{f(x+h) - f(x)}{h}; \hspace{0.5cm}h>0$.

We can use forward or backwards differencing ($+h$ or $-h$), but a good option is usually central differences:

$f'(x)\approx\frac{f(x+\frac{h}{2}) - f(x-\frac{h}{2})}{h}$.

Since we have to evaluate the function at two points anyway, this does not come at a significantly higher computational cost either.

##### BONUS EXERCISE

Implement the Newton method on the Solow model again, but now use finite difference approximations instead of the analytical derivative. I.e. You do not use the function `k_doubleprime()`, but only the differences in values of `k_prime()` to approximate derivatives.